In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib
import seaborn as sns
import nltk
import string
from nltk.corpus import stopwords
from sklearn.metrics import jaccard_score
from nltk.metrics import jaccard_distance
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

# Performing EDA: Exploratory Data Analysis

In [2]:
#cur_dir = Path().resolve()
#print(cur_dir)
#path_to_file = Path.joinpath(cur_dir, 'potential_talents_OG.xlsx')
#join_path = cur_dir / 'potential_talents_OG.xlsx'
#print("the joined path =", join_path)

data = pd.read_excel("potential_talents_OG.xlsx")

In [3]:
# data = pd.read_excel(path_to_file)
# print(data)
print(data)
data.head()

      id                                          job_title  \
0      1  2019 C.T. Bauer College of Business Graduate (...   
1      2  Native English Teacher at EPIK (English Progra...   
2      3              Aspiring Human Resources Professional   
3      4             People Development Coordinator at Ryan   
4      5    Advisory Board Member at Celal Bayar University   
..   ...                                                ...   
99   100  Aspiring Human Resources Manager | Graduating ...   
100  101              Human Resources Generalist at Loparex   
101  102   Business Intelligence and Analytics at Travelers   
102  103                     Always set them up for Success   
103  104   Director Of Administration at Excellence Logging   

                                location connection  fit  
0                         Houston, Texas         85  NaN  
1                                 Kanada      500+   NaN  
2    Raleigh-Durham, North Carolina Area         44  NaN  
3      

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
data.describe() # the stats of the data, mean, mediun, max and min, and std value of the data

,id,fit
count,104.000000,0.0
mean,52.500000,NaN
std,30.166206,NaN
min,1.000000,NaN
25%,26.750000,NaN
50%,52.500000,NaN
75%,78.250000,NaN
max,104.000000,NaN


In [5]:
data.shape # checking the shape of the data

# balance data

(104, 5)

In [6]:
data['job_title']

0      2019 C.T. Bauer College of Business Graduate (...
1      Native English Teacher at EPIK (English Progra...
2                  Aspiring Human Resources Professional
3                 People Development Coordinator at Ryan
4        Advisory Board Member at Celal Bayar University
                             ...                        
99     Aspiring Human Resources Manager | Graduating ...
100                Human Resources Generalist at Loparex
101     Business Intelligence and Analytics at Travelers
102                       Always set them up for Success
103     Director Of Administration at Excellence Logging
Name: job_title, Length: 104, dtype: object

In [7]:
data['location']

0                           Houston, Texas
1                                   Kanada
2      Raleigh-Durham, North Carolina Area
3                            Denton, Texas
4                           İzmir, Türkiye
                      ...                 
99                Cape Girardeau, Missouri
100    Raleigh-Durham, North Carolina Area
101             Greater New York City Area
102               Greater Los Angeles Area
103                            Katy, Texas
Name: location, Length: 104, dtype: object

In [8]:
#To lemmatize and stem an Excel file, remove stop words, remove punctuation, and tokenize the 
#text using RegexpTokenizer, you can follow these steps:

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenize the text
    tokens = tokenizer.tokenize(text) if isinstance(text, str) else text
    
    # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]

    # Remove stop words
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stem
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

input_text = input("Insert your free text: ")
preprocessed_input = preprocess_text(input_text) # Replace with the actual column names to process
preprocessed_data_excel = data['job_title'].apply(preprocess_text)

print(preprocessed_data_excel.to_excel('preprocessed_file.xlsx', index=False))
print(preprocessed_data_excel) #printing out the processed columns
print(preprocessed_input) # printing out the processed input text 

Insert your free text: profession is to be a developer
None
0      [c, bauer, colleg, busi, graduat, magna, cum, ...
1      [nativ, english, teacher, epik, english, progr...
2                    [aspir, human, resourc, profession]
3                        [peopl, develop, coordin, ryan]
4       [advisori, board, member, celal, bayar, univers]
                             ...                        
99     [aspir, human, resourc, manag, graduat, may, s...
100                [human, resourc, generalist, loparex]
101                     [busi, intellig, analyt, travel]
102                                [alway, set, success]
103                    [director, administr, excel, log]
Name: job_title, Length: 104, dtype: object
['profess', 'develop']


In [9]:
df = pd.read_excel('preprocessed_file.xlsx')
df['preprocessed_column'] = df['job_title'].apply(preprocess_text)

df['fit_score'] = df['preprocessed_column'].apply(lambda x: len(set(x).intersection(set(preprocessed_input))) / len(set(x).union(set(preprocessed_input))))
df.to_excel('output_file.xlsx', index=False)
print(df)

                                             job_title  \
0    ['c', 'bauer', 'colleg', 'busi', 'graduat', 'm...   
1    ['nativ', 'english', 'teacher', 'epik', 'engli...   
2          ['aspir', 'human', 'resourc', 'profession']   
3              ['peopl', 'develop', 'coordin', 'ryan']   
4    ['advisori', 'board', 'member', 'celal', 'baya...   
..                                                 ...   
99   ['aspir', 'human', 'resourc', 'manag', 'gradua...   
100      ['human', 'resourc', 'generalist', 'loparex']   
101           ['busi', 'intellig', 'analyt', 'travel']   
102                        ['alway', 'set', 'success']   
103          ['director', 'administr', 'excel', 'log']   

                                   preprocessed_column  fit_score  
0    [c, bauer, colleg, busi, graduat, magna, cum, ...   0.076923  
1    [nativ, english, teacher, epik, english, progr...   0.000000  
2                     [aspir, human, resourc, profess]   0.200000  
3                      [peopl, 